In [3]:
## CONFIG 
from datetime import datetime


naob_file = "NAOB 2001-05.xlsx"
missing_file = "NAOB_konkordans/NAOB 2001-05 mangler.xlsx"

today = datetime.today().strftime("%Y-%m-%d")
outputfile = f'NAOB_2001-2005_med_URN_{today}.xlsx'


In [4]:
## String formatting and parsing functions
from typing import Iterable
import re


def parse_publication(pub_string: str) -> dict:
    """Parse a publication string into a dictionary. 
    
    The publication string is expected to be in the format:
        'Forfatter – Tittel: Undertittel (Sted År)'
        
    where 'År' is the year of publication in 4 digits (YYYY).
    """
    match = re.match(r'(.*?) – (.*?) \((.*?) (\d{4})\)', pub_string)
    if match:
        forfatter = match.group(1)
        full_tittel = match.group(2)
        sted = match.group(3)
        år = match.group(4)
        
        # Del opp tittel og undertittel ved siste forekomst av ': '
        tittel_deler = full_tittel.rsplit(': ', 1)
        tittel = tittel_deler[0].strip()
        undertittel = tittel_deler[1].strip() if len(tittel_deler) > 1 else None
        
        return {
            'Forfatter': forfatter,
            'Tittel': tittel,
            'Undertittel': undertittel,
            'Sted': sted,
            'År': år
        }
    return {}


def replace_non_alpha(string: str, strict: bool = False) -> str:
    """Replace non-alphanumeric punctuation marks with whitespace in the string."""
    if (string is None) or (not string) or (isinstance(string, float)):
        return None
    if isinstance(string, int):
        return str(string)
    if not string.isalnum():
        iterable_string = string.split() if strict else string

        for x in iterable_string: 
            string = string.replace(x, " ") if not x.isalnum() else string
        #print(string)
    return string.strip() 


def flip_names(author: str) -> str: 
    return " ".join(reversed(author.strip().split(" , ")))


def remove_author_annotation(author: str) -> str: 
    return author.replace(" (red.)", "").replace(" et al.", "")


def remove_initials(author: str) -> str:
    if author is None:
        return None
    return " ".join([x for x in author.split() if len(x) > 1])


def split_authors(authors: str, sep="/") -> Iterable[str]:
    if authors is None:
        return None
    return [auth.strip() for auth in authors.split(" " + sep + " ")]


def extract_author(authors: str, sep="/") -> Iterable[str]:
    try: 
        authorlist = split_authors(authors, sep)
        first_author = authorlist[0]
        return first_author
    except TypeError: 
        return None
    except AttributeError: 
        return None


def format_authors_for_comparison(authors: str) -> str: 
    """Format authors from dhlab.Corpus to compare with the NAOB data."""
    return [
        flip_names(auth).strip() for auth in split_authors(authors)
    ]


In [380]:
# Parse metadata from NAOB and query NB API to get URNs and persistent URLs
import pandas as pd
import  dhlab as dh
import dhlab.api.dhlab_api as api
from datetime import datetime


# Les Excel-arket
df = pd.read_excel(naob_file, names=["Publikasjon", "Årstall"])

# Anvend parsing-funksjonen på hver rad
df = pd.concat([df, df['Publikasjon'].apply(parse_publication).apply(pd.Series)], axis=1)

# Formater strengverdiene før spørring
query_df = df.copy()

# Fjern non-alfanumeriske tegn fra tittel
query_df["Tittel"] = df.Tittel.apply(replace_non_alpha)

# Fjern annotasjoner som angir flere forfattere/ roller
query_df["Forfatter"] = query_df.Forfatter.str.replace(" (red.)", "").replace(" et al.", "")

# Fjern non-alfanumeriske tegn fra forfatter
query_df["Forfatter"] = query_df.Forfatter.apply(replace_non_alpha)

# FJern initialer fra forfatter
query_df["Forfatter"] = query_df.Forfatter.apply(remove_initials)

query_df["Førsteforfatter"] = query_df.Forfatter.apply(extract_author, args=("og",))


def get_corpus_from_metadata(row):
    try:
        results = api.document_corpus(
            author=row['Førsteforfatter'],  #Søk bare på første forfatter hvis det er flere
            title=row['Tittel'],
            from_year = int(row['Årstall']),
            to_year = int(row['Årstall']) + 1,
            doctype = "digibok",
        )
    except Exception as e:
        print(f"Failed to query metadata for row {row.name}, {row['Tittel']} by {row['Forfatter']}: {e}")
        #raise e
        results = pd.DataFrame()
    return results


for idx, row in df.iterrows():
    query_row = query_df.loc[idx]
    metadata = get_corpus_from_metadata(query_row)

    if not metadata.empty:
        #urn = find_best_match(row, metadata)
        urn = metadata.iloc[0].get('urn', '')
        df.loc[idx, 'URN'] = urn
        df.loc[idx, 'URL'] = 'https://urn.nb.no/' + df.loc[idx, 'URN']
        
#df.URL = df.URN.map(lambda x: f"https://urn.nb.no/{x}" if x else None)

# Lagre resultatet til en ny Excel-fil
outputfile = f'NAOB_2001-2005_med_URN_{datetime.today().strftime("%Y-%m-%d")}.xlsx'
df.to_excel(outputfile, index=False)

In [382]:
# Sjekk at forfattere med "og" er splittet riktig
#query_df[query_df.Forfatter.fillna("").str.contains("og")]

In [383]:
# Se på de radene som mangler forfatter -> disse må sjekkes
df.loc[df.Forfatter.isna()]

,Publikasjon,Årstall,Forfatter,Tittel,Undertittel,Sted,År,URN,URL
107,Festbok til Olav Angell i anledning av 70-årsd...,2002,NaN,NaN,NaN,NaN,NaN,URN:NBN:no-nb_digibok_2011041108145,https://urn.nb.no/URN:NBN:no-nb_digibok_201104...
154,"Naturgressboka : bygging, drift, vedlikehold a...",2002,NaN,NaN,NaN,NaN,NaN,URN:NBN:no-nb_digibok_2009021804131,https://urn.nb.no/URN:NBN:no-nb_digibok_200902...
235,Metallurgisk ordbok (Oslo 2003) 2. utg. (1. ut...,2003,NaN,NaN,NaN,NaN,NaN,URN:NBN:no-nb_digibok_2009080404056,https://urn.nb.no/URN:NBN:no-nb_digibok_200908...
371,Grovvitser (Oslo 2005) utgitt som bilag til Vi...,2005,NaN,NaN,NaN,NaN,NaN,URN:NBN:no-nb_digibok_2012121005055,https://urn.nb.no/URN:NBN:no-nb_digibok_201212...


In [384]:
# Hent ut klikkbare lenker til de radene som mangler forfatter
df.loc[df.Forfatter.isna()].URL.to_list()

# ingen av dem stemmer overens med NAOB-tekstene, så vi må fylle inn manuelt

['https://urn.nb.no/URN:NBN:no-nb_digibok_2011041108145',
 'https://urn.nb.no/URN:NBN:no-nb_digibok_2009021804131',
 'https://urn.nb.no/URN:NBN:no-nb_digibok_2009080404056',
 'https://urn.nb.no/URN:NBN:no-nb_digibok_2012121005055']

In [385]:
# Load a list of titles missing URNs from the last corpus batch (80)
missing = pd.read_excel("NAOB 2001-05 mangler.xlsx").rename(columns={"Kolonne A": "Publikasjon", "År": "Årstall"})

# Merge the two lists (No new titles in the missing list, length is 421)
fillgaps = df.merge(missing, how='left', on=["Publikasjon", "Årstall"])

fillgaps[(fillgaps.Grunn == "Mer enn én forfatter") & fillgaps.URL.isna()]
# Gir 1 treff: Christopher Friis-Baastad Grøndahl og Arne Svingen - Ayatollah highway

print(f"{len(fillgaps[fillgaps.URL.isna()])} titler har ikke fått URL.")

fillgaps.to_excel("NAOB_2001-2005_med_URN_og_mangler.xlsx", index=False)


36 titler har ikke fått URL.


In [391]:
# Se på radene som mangler URL
fillgaps[fillgaps.URL.isna()]

,Publikasjon,Årstall,Forfatter,Tittel,Undertittel,Sted,År,URN,URL,Grunn
0,"Anne Enger Lahnstein – Grønn dame, rød klut : ...",2001,Anne Enger Lahnstein,"Grønn dame, rød klut",erindringer,Oslo,2001,NaN,NaN,"Bare ""Enger"" i Nettbibl."
11,Brynjulf Raaen – Den som brenner får svi : en ...,2001,Brynjulf Raaen,Den som brenner får svi,en røverhistorie,Oslo,2001,NaN,NaN,NaN
21,Frode Øverli – Pondus. Første omgang (Oslo 2001),2001,Frode Øverli,Pondus. Første omgang,None,Oslo,2001,NaN,NaN,NaN
26,Hans-Wilhelm Steinfeld – Frihetens bitre tiår ...,2001,Hans-Wilhelm Steinfeld,Frihetens bitre tiår,None,Oslo,2001,NaN,NaN,NaN
63,Sissel Lange-Nielsen – Rød engel : en russisk ...,2001,Sissel Lange-Nielsen,Rød engel,en russisk roman,Oslo,2001,NaN,NaN,NaN
67,Sverre Knudsen – Spille for stjernene : roman ...,2001,Sverre Knudsen,Spille for stjernene,roman,Oslo,2001,NaN,NaN,NaN
87,Anne B. Ragde – Dr. Zellwegers gave : roman (O...,2002,Anne B. Ragde,Dr. Zellwegers gave,roman,Oslo,2002,NaN,NaN,NaN
100,Christopher Friis-Baastad Grøndahl og Arne Svi...,2002,Christopher Friis-Baastad Grøndahl og Arne Svi...,Ayatollah highway,None,Oslo,2002,NaN,NaN,Mer enn én forfatter
102,Dag Solstad – 16.07.41 (Oslo 2002),2002,Dag Solstad,16.07.41,None,Oslo,2002,NaN,NaN,NaN
105,Espen Hammer – Theodor W. Adorno (Oslo 2002),2002,Espen Hammer,Theodor W. Adorno,None,Oslo,2002,NaN,NaN,NaN


Jeg har gått gjennom listen manuelt, søkt opp tittel og/eller forfatter på nb.no og kopiert permanent lenke med URN. 

Lagret med nytt navn: `"NAOB_2001-2005_med_URN_og_mangler_manuelt_redigert.xlsx"`

In [24]:
import pandas as pd
# Last inn manuelt redigerte data
manual_fixes = "NAOB_2001-2005_med_URN_og_mangler_manuelt_redigert.xlsx"

cols = "Publikasjon	Årstall	Forfatter	Tittel	Undertittel	Sted	URN	URL	Grunn".split()
df = pd.read_excel(manual_fixes, names=cols).drop(index=105) # Fjern raden med kolonnenavnene

print("Antall URNer som er riktig, starter på URN. Resten er de manuelt kopierte URL-ene som starter på https \n")
print('Starter med  "URN:NBN:no": ', df.URN.str.startswith("URN:NBN:no").value_counts(), "\n")
print('Starter med "https": ', df.URN.str.startswith("https").value_counts())

# Fjern https-prefixet fra URN-kolonnen
df.URN = df.URN.str.replace("https://urn.nb.no/", "")

# Hent et dhlab.Corpus-objekt fra URNene
valid_urns = df.URN[df.URN.str.startswith("URN")]


valid_urns


Antall URNer som er riktig, starter på URN. Resten er de manuelt kopierte URL-ene som starter på https 

Starter med  "URN:NBN:no":  URN
True     417
False      2
Name: count, dtype: int64 

Starter med "https":  URN
False    419
Name: count, dtype: int64


0      URN:NBN:no-nb_digibok_2011070808013
1      URN:NBN:no-nb_digibok_2011071808080
2      URN:NBN:no-nb_digibok_2011060106087
3      URN:NBN:no-nb_digibok_2014082107036
4      URN:NBN:no-nb_digibok_2011070806005
                      ...                 
413    URN:NBN:no-nb_digibok_2011100406098
414    URN:NBN:no-nb_digibok_2008112700079
415    URN:NBN:no-nb_digibok_2011032306166
417    URN:NBN:no-nb_digibok_2012052105045
418    URN:NBN:no-nb_digibok_2009021904016
Name: URN, Length: 417, dtype: object

In [23]:
new = valid_urns.to_frame()
#new.drop(index=106, inplace=True)
new["corpus"] = "naob_2024"
new.rename(columns={"URN": "urn"}, inplace=True)

naob_corpus = pd.read_csv("naob.csv", index_col=0)
pd.concat([naob_corpus, new]).to_csv("naob.csv")

In [259]:
# IKKE FERDIG - MÅ FIKSE LOGIKKEN 

# Funksjon for å finne beste treff
def find_best_match(row, metadata):
    
    author=row['Forfatter']
    title=row['Tittel']
    year = int(row['Årstall'])
    
    
#    search_string = row['Søkestreng'].lower()
    best_match = None
    best_score = 0
    
    for idx, item in metadata.iterrows():
        dh_title = item.get('title', '').lower()
        authors = ' '.join(item.get('authors', [])).lower()
        year = str(item.get('year', ''))
        
        #item_string = f"{title} {authors} {year}"
        
        # Enkel poengberegning basert på hvor mange ord som matcher
        score = sum(1 for word in search_string.split() if word in item_string)
        
        if score > best_score:
            best_score = score
            best_match = item
    
    return best_match['urn'] if best_match else ''




In [ ]:

# AUTOGENERERT KODE FRA CLAUDE 3.5 Sonnet 

metadata = corpus

# Funksjon for å finne beste treff
def find_best_match(row, metadata):
    search_string = row['Søkestreng'].lower()
    best_match = None
    best_score = 0
    
    for item in metadata:
        title = item.get('title', '').lower()
        authors = ' '.join(item.get('authors', [])).lower()
        year = str(item.get('year', ''))
        
        item_string = f"{title} {authors} {year}"
        
        # Enkel poengberegning basert på hvor mange ord som matcher
        score = sum(1 for word in search_string.split() if word in item_string)
        
        if score > best_score:
            best_score = score
            best_match = item
    
    return best_match['urn'] if best_match else ''

# Finn URN for hver publikasjon
df['URN'] = df.apply(lambda row: find_best_match(row, metadata), axis=1)

# Lag fullstendig URL
df['URL'] = 'https://urn.nb.no/' + df['URN']

# Lagre resultatet til en ny Excel-fil
df.to_excel('publikasjoner_med_url.xlsx', index=False)


In [392]:
## Test parse_publication
examples = [
    "Forfatter – Hovedtittel: Undertittel (Oslo 2001)",
    "Forfatter1 og Forfatter2 – Hovedtittel: Undertittel (Oslo 2001)",
    "Forfatter – Kompleks tittel: med flere: kolon: Faktisk undertittel (Oslo 2001)",
    "Forfatter – Tittel uten undertittel (Oslo 2001)"
]

for example in examples:
    result = parse_publication(example)
    print(f"Original: {example}")
    print(f"Parsed: {result}\n")


Original: Forfatter – Hovedtittel: Undertittel (Oslo 2001)
Parsed: {'Forfatter': 'Forfatter', 'Tittel': 'Hovedtittel', 'Undertittel': 'Undertittel', 'Sted': 'Oslo', 'År': '2001'}

Original: Forfatter1 og Forfatter2 – Hovedtittel: Undertittel (Oslo 2001)
Parsed: {'Forfatter': 'Forfatter1 og Forfatter2', 'Tittel': 'Hovedtittel', 'Undertittel': 'Undertittel', 'Sted': 'Oslo', 'År': '2001'}

Original: Forfatter – Kompleks tittel: med flere: kolon: Faktisk undertittel (Oslo 2001)
Parsed: {'Forfatter': 'Forfatter', 'Tittel': 'Kompleks tittel: med flere: kolon', 'Undertittel': 'Faktisk undertittel', 'Sted': 'Oslo', 'År': '2001'}

Original: Forfatter – Tittel uten undertittel (Oslo 2001)
Parsed: {'Forfatter': 'Forfatter', 'Tittel': 'Tittel uten undertittel', 'Undertittel': None, 'Sted': 'Oslo', 'År': '2001'}

